## Encoders

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU,Conv2D, BatchNormalization, Flatten, InputLayer, concatenate, ReLU
from tensorflow.keras.layers import Conv2DTranspose,Input,Dense,Reshape, Activation, Flatten, Concatenate
from tensorflow.keras.models import Sequential
from encoders import *
from noises import *
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar100, cifar10

In [2]:
(train_data_clean, _), (test_data_clean, _) = cifar100.load_data()
train_data_clean = train_data_clean.astype('float32')/255.
train_data_noisy = np.zeros(train_data_clean.shape)
for i in range(train_data_clean.shape[0]):
    train_data_noisy[i] =  normal_noise(train_data_clean[i])
test_data_noisy = np.zeros(test_data_clean.shape)
for i in range(test_data_clean.shape[0]):
    test_data_noisy[i] =  normal_noise(test_data_clean[i])

### Simple U- Net autoencoder

In [3]:
import datetime
import tensorflow as tf
import os
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

modelU = UNetModel(32,256)
modelU.compile(loss= 'mean_squared_error', optimizer=tf.keras.optimizers.Adam(clipnorm=1), metrics=['accuracy'])
#encoded = modelU.encode(train_data_clean[:10])

In [4]:
Unet_history = modelU.fit(train_data_noisy,train_data_clean, batch_size=64,verbose=1, epochs=40, validation_split=0.15, callbacks=[ callback])


Epoch 1/40


InternalError:  No MLCTrainingGraph has been found.
	 [[node gradient_tape/mean_squared_error/MLCMeanSquaredErrorGrad (defined at /Users/smaket/miniforge3/envs/tf24/lib/python3.8/site-packages/tensorflow/compiler/tf2mlcompute/ops/gen_mlc_ops.py:5883) ]]
	 [[MLCSubgraphOp_0_6]] [Op:__inference_train_function_4863]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/mean_squared_error/MLCMeanSquaredErrorGrad:
 u_net_model/StatefulPartitionedCall (defined at <ipython-input-4-2236102244f1>:1)	
 mean_squared_error/MLCMeanSquaredError (defined at /Users/smaket/miniforge3/envs/tf24/lib/python3.8/site-packages/tensorflow/compiler/tf2mlcompute/ops/gen_mlc_ops.py:5801)

Function call stack:
train_function


In [ ]:
plt.plot(Unet_history.history['loss'])

In [ ]:
modelUNoDense = UNetModel(32,256, True)
modelUNoDense.compile(loss= 'mean_squared_error', optimizer=tf.keras.optimizers.Adam(clipnorm=1), metrics=['accuracy'])

UneNoDenset_history = modelUNoDense.fit(train_data_noisy,train_data_clean, batch_size=64,verbose=1, epochs=40, validation_split=0.15, callbacks=[ callback])


### U - net autoencoder with dense layer 

### Cusomized Simple Autoencoder

In [ ]:

model = SimpleDAE(256*2*2)
model.compile(loss = 'mse', optimizer='adam', metrics=['accuracy'])

Simple_DEA_history = model.fit(train_data_noisy,train_data_clean, batch_size=64,verbose=1, epochs=40, validation_split=0.15,callbacks=[callback])



In [ ]:
predU = modelU.predict(train_data_noisy[:10])
fig,ax = plt.subplots(3,3)
ax[0,0].imshow(predU[2])
ax[0,1].imshow(train_data_noisy[2])
ax[0,2].imshow(train_data_clean[2])


predUnodense = modelUNoDense.predict(train_data_noisy[:10])

ax[1,0].imshow(predUnodense[2])
ax[1,1].imshow(train_data_noisy[2])
ax[1,2].imshow(train_data_clean[2])

#%%

pred = model.predict(train_data_noisy[:10])

ax[2,0].imshow(pred[2])
ax[2,1].imshow(train_data_noisy[2])
ax[2,2].imshow(train_data_clean[2])

In [ ]:
def build_encoder(encoded_dimension):
    model = Sequential([

    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(encoded_dimension, activation='relu')
    ])
    return model
def build_decoder():
    model = Sequential([
    Dense(2 * 2 * 256, activation='relu'),
    Reshape((2, 2, 256)),

    Conv2DTranspose(filters=256, kernel_size=3,strides=1, padding='same'),
    ReLU(),
    BatchNormalization(),
    Conv2DTranspose(filters=128, kernel_size=3,strides=2, padding='same'),
    ReLU(),
    BatchNormalization(),
    Conv2DTranspose(filters=64, kernel_size=3,strides=2, padding='same'),
    ReLU(),
    BatchNormalization(),
    Conv2DTranspose(filters=32, kernel_size=3,strides=2, padding='same'),
    ReLU(),
    BatchNormalization(),
    Conv2DTranspose(filters=3, kernel_size=3,strides=2, padding='same'),
    Activation('sigmoid')
    ])
    return model
class SimpleDAEpadd2(tf.keras.Model):
    def __init__(self,encoded_dimension):
        super().__init__()


        self.encoder = build_encoder(encoded_dimension)
        self.decoder = build_decoder()
    def call(self,x):
        x = self.encoder(x)
        return self.decoder(x)

In [ ]:
modelpadd2 = SimpleDAEpadd2(256*2*2)
modelpadd2.compile(loss = 'mse', optimizer='adam', metrics=['accuracy'])


#%%

padd2_history = modelpadd2.fit(train_data_noisy,train_data_clean, batch_size=64,verbose=1, epochs=40, validation_split=0.15, callbacks=[callback])



In [ ]:
pred2padd = modelpadd2.predict(train_data_noisy[:10])
fig,ax = plt.subplots(1,3)
i = np.random.choice([x for x in range(10)])
ax[2].imshow(pred2padd[i])
ax[1].imshow(train_data_noisy[i])
ax[0].imshow(train_data_clean[i])

In [ ]:
from matplotlib.ticker import FormatStrFormatter

fig,ax = plt.subplots(2,4,figsize = (20,10))
for i in range(2):
  for j in range(4):
    ax[i,j].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[i,j].set_xlabel('epoch')
    ax[i,j].set_ylabel('loss') if i == 0 else ax[i,j].set_ylabel('accuracy')
ax[0,0].plot([i for i in range(40)],np.array([UneNoDenset_history.history['loss'], UneNoDenset_history.history['val_loss']]).T)
ax[0,0].set_title('U-net  with dense layer loss')
ax[0,0].legend(['loss','validation_loss'])
ax[0,1].plot([i for i in range(40)],np.array([Unet_history.history['loss'], Unet_history.history['val_loss']]).T)
ax[0,1].set_title('U-net without dense layer loss')
ax[0,1].legend(['loss','validation_loss'])
ax[0,2].plot([i for i in range(40)],np.array([Simple_DEA_history.history['loss'], Simple_DEA_history.history['val_loss']]).T)
ax[0,2].set_title('Simple Autoencoder  loss')
ax[0,2].legend(['loss','validation_loss'])
ax[0,3].plot([i for i in range(40)],np.array([padd2_history.history['loss'], padd2_history.history['val_loss']]).T)
ax[0,3].set_title('Simple Autoencoder with double transposition loss')
ax[0,3].legend(['loss','validation_loss'])
ax[1,0].plot([i for i in range(40)],np.array([UneNoDenset_history.history['accuracy'], UneNoDenset_history.history['val_accuracy']]).T)
ax[1,0].set_title('U-net  with dense layer accuracy')
ax[1,0].legend(['loss','validation_loss'])
ax[1,1].plot([i for i in range(40)],np.array([Unet_history.history['accuracy'], Unet_history.history['val_accuracy']]).T)
ax[1,1].set_title('U-net  without dense layer accuracy')
ax[1,1].legend(['loss','validation_loss'])
ax[1,2].plot([i for i in range(40)],np.array([Simple_DEA_history.history['accuracy'], Simple_DEA_history.history['val_accuracy']]).T)
ax[1,2].set_title('Simple Autoencoder accuracy')
ax[1,2].legend(['loss','validation_loss'])
ax[1,3].plot([i for i in range(40)],np.array([padd2_history.history['accuracy'], padd2_history.history['val_accuracy']]).T)
ax[1,3].set_title('Simple Autoencoder with double transposition  accuracy')
ax[1,3].legend(['loss','validation_loss'])
